## Subtyping Mitotic Figures (MF) and Mitotic-Like Figures (MLFs) 

I. Imports

In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.layers import Dense
from tensorflow.keras import optimizers
import numpy as np
from numpy.core.multiarray import packbits
import pandas as pd
import os
from os import listdir
import cv2
from google.colab.patches import cv2_imshow
from matplotlib import image as mpimg
from matplotlib import pyplot as plt
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
#This cell is only relevant if using Google Colab and Google Drive.  It should not be run unless running this Jupyter Notebook in Google Colab.
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
#This cell is only relevant if using Google Colab and Google Drive.  It should not be run unless running this Jupyter Notebook in Google Colab.
%cd drive/MyDrive

/content/drive/MyDrive


II. Data Upload and Data Cleaning

In [ ]:
#Here, we read in FSL annotations and concatenate them into a single Pandas dataframe.
data1 = pd.read_csv('fsl_subclassing/slide_2_updated/20221116-115200_ann_df_re-subcl_slide_no_2_all_64_MFs.csv')
data2 = pd.read_csv('fsl_subclassing/slide_2/20221030-011825_annot_df_sorted_subclassed_MLFs_32_slide_no_2.csv')
data3 =  pd.read_csv('fsl_subclassing/slide_3/20221112-015900_annot_df_sorted_subclassed_MLFs_72_slide_no_3.csv')
data4 =  pd.read_csv('fsl_subclassing/slide_3/20221112-003547_annot_df_sorted_subclassed_slide_no_3_all_MFs.csv')
data5 = pd.read_csv('fsl_subclassing/slide_5/20221112-230321_annot_df_sorted_subclassed_MLFs_100_slide_no_5.csv')
data6 = pd.read_csv('fsl_subclassing/slide_5/221211-35722_ann_df_subcl_CORRECTED_sl_5_all_130_MFs.csv')
data7 = pd.read_csv('fsl_subclassing/slide_7/20221115-150013_annot_df_sorted_subclassed_MLFs_160_slide_no_7.csv')
data8 = pd.read_csv('fsl_subclassing/slide_7/20221115-150013_ann_df_subcl_slide_no_7_all_157_MFs.csv')
data9 = pd.read_csv('fsl_subclassing/slide_8/20221116-092800_ann_df_subcl_slide_no_8_all_187_MFs.csv')
data10 = pd.read_csv('fsl_subclassing/slide_8/20221116-092800_annot_df_sorted_subclassed_MLFs_100_slide_no_8.csv')
data = pd.concat([data1, data2, data3, data4, data5, data6, data7, data8, data9,data10])


In [ ]:
#This cell is only relevant if using Google Colab and Google Drive.  It should not be run unless running this Jupyter Notebook in Google Colab.
%cd MITOS_WSI_CMC/databases

/content/drive/MyDrive/MITOS_WSI_CMC/databases


In [ ]:
#The FSL annotations have labels 9 and 42, which are equivalent, and 0, none of which should be used for training.  We drop these.
data.head()
data['subcl'] = data['subcl'].replace(42,9)
data_filtered = data[data['subcl'] != 9]
data = data_filtered.copy()
data = data[data['subcl'] != 0]

In [ ]:
#The FSL subclass annotations extracted from the dataframe to train on.
subclass_annotations = data.drop(['ctr', 'coord', 'guid'], axis = 1)
subclass_annotations.head()
len(subclass_annotations)

984

In [ ]:
#Here, we load the images from the files we wrote during the patch extraction step.


labels = subclass_annotations['subcl']
imagelist = []

# get the path/directory
folder_dir = "databases"
i = 0

#We use a loop to read in images from each slide and name them with a filename convention that includes the slide in the ID.
while (i<93):
        temp = mpimg.imread('/content/drive/MyDrive/MITOS_WSI_CMC/databases/patchExtraction_final/' + str(2) + str(i) + str('.png'))
        imagelist.append(temp)
        i = i + 1
i = 0
while (i<155):
        temp = mpimg.imread('/content/drive/MyDrive/MITOS_WSI_CMC/databases/patchExtraction_final/' + str(3) + str(i) + str('.png'))
        imagelist.append(temp)
        i = i + 1
i = 0
while (i<203):
        temp = mpimg.imread('/content/drive/MyDrive/MITOS_WSI_CMC/databases/patchExtraction_final/' + str(5) + str(i) + str('.png'))
        imagelist.append(temp)
        i = i + 1
i = 0
while (i<303):
        temp = mpimg.imread('/content/drive/MyDrive/MITOS_WSI_CMC/databases/patchExtraction_final/' + str(7) + str(i) + str('.png'))
        imagelist.append(temp)
        i = i + 1
i = 0
while (i<230):
        temp = mpimg.imread('/content/drive/MyDrive/MITOS_WSI_CMC/databases/patchExtraction_final/' + str(8) + str(i) + str('.png'))
        imagelist.append(temp)
        i = i + 1


In [ ]:
#Here, we collapse labels 3,4, and 5 into just label 5.  
labels = np.asarray(labels)
i = 0
while i<len(labels):
    if (np.equal(int(labels[i]),3)):
       labels[i] = 5
    elif (np.equal(int(labels[i]),4)):
       labels[i] = 5
    i = i + 1


In [ ]:
#Labels are manually one-hot encoded.
i = 0
ohe_labels = [0] * len(labels)
while (i<len(labels)):
      if (labels[i] == 5):
         ohe_labels[i] = [0, 0, 0, 1]
      elif (labels[i] == 6):
         ohe_labels[i] = [0, 0, 1, 0]
      elif (labels[i] == 7):
         ohe_labels[i] = [0, 1, 0, 0]
      elif (labels[i] == 8):
         ohe_labels[i] = [1, 0, 0, 0]
      i = i + 1

In [ ]:
#Labels and images are put into "X" and "y" for the train-test split step.
X = imagelist.copy()
X = np.asarray(X)
y = np.asarray(ohe_labels)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=42)

III. VGG16 for Classification

In [ ]:
#VGG16 is initialized.

vgg_model = tf.keras.applications.VGG16(include_top=False,weights="imagenet",input_shape=(64,64,3),classifier_activation="relu")




58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
#The sizes of the dense layers were chosen as multiples of the number of classes.
model = models.Sequential()
model.add(vgg_model)
model.add(layers.Flatten())
model.add(layers.Dense(4, activation='softmax'))


In [ ]:
model.compile(optimizer=optimizers.SGD(learning_rate=0.0001,momentum=0.0,nesterov=False,name='SGD'), loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train, batch_size=32, epochs=40, verbose=1, validation_data=(X_test , y_test))

Epoch 1/40
22/22 [==============================] - 142s 6s/step - loss: 1.4188 - accuracy: 0.3765 - val_loss: 1.3548 - val_accuracy: 0.4459
Epoch 2/40
22/22 [==============================] - 136s 6s/step - loss: 1.2731 - accuracy: 0.4622 - val_loss: 1.2931 - val_accuracy: 0.4628
Epoch 3/40
22/22 [==============================] - 140s 6s/step - loss: 1.2342 - accuracy: 0.4680 - val_loss: 1.2701 - val_accuracy: 0.4628
Epoch 4/40
22/22 [==============================] - 138s 6s/step - loss: 1.2149 - accuracy: 0.4753 - val_loss: 1.2574 - val_accuracy: 0.4595
Epoch 5/40
22/22 [==============================] - 137s 6s/step - loss: 1.2026 - accuracy: 0.4738 - val_loss: 1.2511 - val_accuracy: 0.4662
Epoch 6/40
22/22 [==============================] - 136s 6s/step - loss: 1.1919 - accuracy: 0.4782 - val_loss: 1.2460 - val_accuracy: 0.4696
Epoch 7/40
22/22 [==============================] - 141s 6s/step - loss: 1.1808 - accuracy: 0.4797 - val_loss: 1.2414 - val_accuracy: 0.4730
Epoch 8/40
22

V. Saving the Model Weights in .h5 format

In [ ]:
model.save('final_model.h5')